In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("./산림복지/2020산림복지_본문_노이즈 제거.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

  0%|          | 0/6415 [00:00<?, ?it/s]

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/naturalperso/222192091578,2020년에도 열심히 살은 나의 삶,2020.12.31.,이제는\n열심히\n부지런히\n최선을\n잘\n봉사 하는 삶을\n내려 놓아도\n잘 산다...,0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,[(이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 ...
1,https://blog.naver.com/abc112212/222192059763,위장환경주의 ; '그린'으로 포장한 기업의 실체,2020.12.31.,"플라스틱 쓰레기 문제, 육식을 위한 가축산업으로 인한 문제, 산림 파괴, 석유 누출...",1,플라스틱 쓰레기 문제 육식을 위한 가축산업으로 인한 문제 산림 파괴 석유 누출로 인...,[(플라스틱 쓰레기 문제 육식을 위한 가축산업으로 인한 문제 산림 파괴 석유 누출로...
2,https://blog.naver.com/yrangal/222191919215,[濟 經],2020.12.31.,석가 공자 예수는 성인이자 현인이기도 하다.\n이 분들은 중생을 구제하는 빼어난 지...,2,석가 공자 예수는 성인이자 현인이기도 하다 이 분들은 중생을 구제하는 빼어난 지혜가...,"[(석가 공자 예수는 성인이자 현인이기도 하다, 0, 23, None, []), (..."
3,https://blog.naver.com/jtwon21/222191901961,2020 통영거제환경연합 올해의 환경뉴스는?,2020.12.31.,2020년 통영거제환경운동연합 선정 ‘올해의 환경 뉴스’\n#통영거제환경운동연합은 ...,3,2020년 통영거제환경운동연합 선정 올해의 환경 뉴스 통영거제환경운동연합은 2020...,[(2020년 통영거제환경운동연합 선정 올해의 환경 뉴스 통영거제환경운동연합은 20...
4,https://blog.naver.com/inhyangin/222191824736,"전남도, 산림행정종합평가... 광양․보성·해남/세계 산림치...",2020.12.31.,"블로그\n[전남] 7,530개의 글\n[전남]\n목록열기\n전남도, 산림행정종합평가...",4,블로그 전남 7 530개의 글 전남 목록열기 전남도 산림행정종합평가 담양군 최우수 ...,[(블로그 전남 7 530개의 글 전남 목록열기 전남도 산림행정종합평가 담양군 최우...
...,...,...,...,...,...,...,...
6410,https://blog.naver.com/nature_school/221756708673,산림교육프로그램 인증 DONE!,2020.01.01.,[홍유릉 자연학교]\n홍유릉 자연학교가\n산림교육프로그램 2가지를\n동시에 인증받았...,6410,홍유릉 자연학교 홍유릉 자연학교가 산림교육프로그램 2가지를 동시에 인증받았습니다 D...,[(홍유릉 자연학교 홍유릉 자연학교가 산림교육프로그램 2가지를 동시에 인증받았습니다...
6411,https://blog.naver.com/inhyangin/221756499038,"광양시, 2020년 1월 월간 업무보고_3 [조순익 기자]",2020.01.01.,"블로그\n[공지/날씨/오늘] 3,834개의 글\n[공지/날씨/오늘]\n목록열기\n광...",6411,블로그 공지 날씨 오늘 3 834개의 글 공지 날씨 오늘 목록열기 광양시 2020년...,[(블로그 공지 날씨 오늘 3 834개의 글 공지 날씨 오늘 목록열기 광양시 202...
6412,https://blog.naver.com/humanism1968/221756321414,(( 주간투자전략 )) 삼성전자 잠정실적(-) 예상으로 시장 및...,2020.01.01.,블로그\n공지 목록\n공지글\n글 제목 작성일\n공지 [ 주식 이야기 ] 시장이야기...,6412,블로그 공지 목록 공지글 글 제목 작성일 공지 주식 이야기 시장이야기 시장흐름 종목...,[(블로그 공지 목록 공지글 글 제목 작성일 공지 주식 이야기 시장이야기 시장흐름 ...
6413,https://blog.naver.com/yneconomic/221756088888,"경북도정, 땀과 열정으로 공고한 성장 기틀 마련",2020.01.01.,"지난 26일, 이철우도지사는 도청에서 기자간담회를 열고, 2019년 도정의 주요성과...",6413,지난 26일 이철우도지사는 도청에서 기자간담회를 열고 2019년 도정의 주요성과와 ...,[(지난 26일 이철우도지사는 도청에서 기자간담회를 열고 2019년 도정의 주요성과...


In [5]:
# 데이터프레임을 100개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129


In [6]:
# df.to_csv("2020산림복지_세이브포인트.csv", index=False)

In [7]:
# df = pd.read_csv("2020산림복지_세이브포인트.csv")
# df

In [8]:
df["split_list"][0]

[Sentence(text='이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충 살고 싶은 2020이었다', start=0, end=60, tokens=None, subs=[]),
 Sentence(text='필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런히 잘 최선을 봉사 오지랖으로 살았다', start=61, end=127, tokens=None, subs=[]),
 Sentence(text='교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제조기능사 교육청 전통음식 재능기부 그만두었는데 3월 군청 홈페이지 교육청 봉사자 모집 고민 지나갔지 4월 군청 홈페이지에 교육청 다시 봉사자 모집 내가 해야 될 봉사 면접까지 하는 봉사 안갈려고 했는데 신청자가 없어서 하게 되었지 ㅋㅋㅋ 봉사라기 보다 일 이었지 ㅋㅋㅋ', start=128, end=324, tokens=None, subs=[]),
 Sentence(text='뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를 생각하게 만든 시간이었다', start=325, end=385, tokens=None, subs=[]),
 Sentence(text='오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑게 맞아 주셨고 만날 때마다 고맙다고 해주셨고 연수에서는 존경한다는 이야기까지 더 열심히 할 수 밖에 없었다', start=386, end=492, tokens=None, subs=[]),
 Sentence(text='칭찬은 고래를 춤추게 하듯 과장님의 따뜻함이 감사했다 교육장님께서도 볼 때마다 고맙다고 따뜻하게 대해주셨다', start=493, end=552, tokens=None, subs=[]),
 Sentence(text='코로나로 어려운 시기에도 부지런히 열심히 12월 마지막 달까지 어려운 이웃을 생각하고 주민복지위원으로도 잘 살았다', sta

In [9]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("./산림복지/2020산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

In [10]:
df2 = pd.read_csv("./산림복지/2020산림복지_본문_전처리.csv")
df2

,split_str,org_idx
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0
...,...,...
181224,그는 앞으로도 초심을 잃지 않고 지금까지 해온 것을 바탕으로 오로지 부자 청송군민을...,6414
181225,이처럼 혜안을 가진 그가 있기에 청송군의 미래 행복은 당연히 밝은 일이 될 것임은 ...,6414
181226,우스갯말로 지금부터 청송군민들은 너도 나도 신나게 노래부르면서 새로운 청송의 역사를...,6414
181227,자세한 내용은 http yne kr news view php idx 6675 mco...,6414


In [11]:
dfs['df_32']['split_list'].index

RangeIndex(start=1550, stop=1600, step=1)

In [12]:
df2

,split_str,org_idx
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0
...,...,...
181224,그는 앞으로도 초심을 잃지 않고 지금까지 해온 것을 바탕으로 오로지 부자 청송군민을...,6414
181225,이처럼 혜안을 가진 그가 있기에 청송군의 미래 행복은 당연히 밝은 일이 될 것임은 ...,6414
181226,우스갯말로 지금부터 청송군민들은 너도 나도 신나게 노래부르면서 새로운 청송의 역사를...,6414
181227,자세한 내용은 http yne kr news view php idx 6675 mco...,6414


In [13]:
a = 129
for i in tqdm(dfs['df_%s'%a]['split_list'].index):
    for j in range(len(dfs['df_%s'%a]['split_list'][i])):
        split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2.to_csv("./산림복지/2020산림복지_본문_전처리.csv", index=False, encoding='UTF-8')
df2

,split_str,org_idx
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0
...,...,...
181931,그는 앞으로도 초심을 잃지 않고 지금까지 해온 것을 바탕으로 오로지 부자 청송군민을...,6414
181932,이처럼 혜안을 가진 그가 있기에 청송군의 미래 행복은 당연히 밝은 일이 될 것임은 ...,6414
181933,우스갯말로 지금부터 청송군민들은 너도 나도 신나게 노래부르면서 새로운 청송의 역사를...,6414
181934,자세한 내용은 http yne kr news view php idx 6675 mco...,6414


In [14]:
df2.drop_duplicates(inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("./산림복지/2020산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

In [15]:
df2

,split_str,org_idx
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0
...,...,...
181224,그는 앞으로도 초심을 잃지 않고 지금까지 해온 것을 바탕으로 오로지 부자 청송군민을...,6414
181225,이처럼 혜안을 가진 그가 있기에 청송군의 미래 행복은 당연히 밝은 일이 될 것임은 ...,6414
181226,우스갯말로 지금부터 청송군민들은 너도 나도 신나게 노래부르면서 새로운 청송의 역사를...,6414
181227,자세한 내용은 http yne kr news view php idx 6675 mco...,6414


## 맞춤법 검사

In [1]:
import pandas as pd
df2 = pd.read_csv("./산림복지/2020산림복지_본문_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0
...,...,...
181224,그는 앞으로도 초심을 잃지 않고 지금까지 해온 것을 바탕으로 오로지 부자 청송군민을...,6414
181225,이처럼 혜안을 가진 그가 있기에 청송군의 미래 행복은 당연히 밝은 일이 될 것임은 ...,6414
181226,우스갯말로 지금부터 청송군민들은 너도 나도 신나게 노래부르면서 새로운 청송의 역사를...,6414
181227,자세한 내용은 http yne kr news view php idx 6675 mco...,6414


In [2]:
#- 패키지 임포트
import requests
import json
from tqdm.notebook import tqdm
import re
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

'''맞춤법 검사 함수'''
def ko_grammar(sent):
    global correct_list ; correct_list= []
    #for i in tqdm(range(len(a))):
    for i in tqdm(range(len(sent))):
        try:
            print(i)                                                                               
            response = requests.post('http://164.125.7.61/speller/results', data={'text1': sent[i]})  # 맞춤법 검사 요청 (requests)    
            data = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0] # 응답에서 필요한 내용 추출 (html 파싱)
            data = json.loads(data) # 파이썬 딕셔너리 형식으로 변환
            orgStr = [err['orgStr'] for err in data['errInfo']] #오류가 담긴 errinfo에서 고쳐야 되는 단어 리스트로 추출 
            correct = [err['candWord'] for err in data['errInfo']] #고쳐진 단어도 리스트로 추출
            splited = sent[i] 

            '''여러 개로 고쳐졌을 때 맨 처음 단어만 선택'''
            for j in range(len(correct)):
                if '|'in correct[j]: #여러 개로 고쳐질 경우 문자열이 '|'로 구분되어 있음
                    com = re.compile('\|')
                    many = com.search(correct[j])
                    correct[j]= correct [j][0:(many.span()[0])]
                '''틀린 문장 고치기'''   
                splited = splited.replace(orgStr[j], correct[j])

        except:      
            splited = sent[i]

        correct_list.append(splited)
        display.clear_output(wait=True) # 출력지우기

Output()

In [3]:
# 데이터프레임을 100개 단위로 나누기
chunk_size = 5000
num_chunks = len(df2) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df2[start_idx:end_idx]
    chunk_df.reset_index(drop=True, inplace=True)
    dfs[f"df2_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df2_1
df2_2
df2_3
df2_4
df2_5
df2_6
df2_7
df2_8
df2_9
df2_10
df2_11
df2_12
df2_13
df2_14
df2_15
df2_16
df2_17
df2_18
df2_19
df2_20
df2_21
df2_22
df2_23
df2_24
df2_25
df2_26
df2_27
df2_28
df2_29
df2_30
df2_31
df2_32
df2_33
df2_34
df2_35
df2_36
df2_37


In [19]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df_s = pd.DataFrame(columns = ['split_str', 'org_idx', 'correct_str']) # 새로운 데이터프레임 제작

# ''' 문장 데이터 프레임 저장 '''
# df_s.to_csv('./산림복지/2020산림복지_본문_맞춤법검사.csv', index=False, encoding='UTF-8')

In [4]:
df_s = pd.read_csv("./산림복지/2020산림복지_본문_맞춤법검사.csv", encoding='UTF-8')
df_s

,split_str,org_idx,correct_str
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...
...,...,...,...
59995,잔금지급일과 등기일 중 빠른날 Q 3주택을 소유한 1세대 중 A가 20 5 15 공...,1974,"잔금 지급일과 등기일 중 빠른 날 Q 3주택을 소유한 1세대 중 A가 20, 5, ..."
59996,Q 20 8 12 전에 취득한 업무용 오피스텔을 20 8 12 이후에 주거용 오피스...,1974,"Q20, 8, 12전에 취득한 업무용 오피스텔을 20, 8, 12 이후에 주거용 오..."
59997,부칙 제3조 주택 수의 판단 범위에 관한 적용례 제13조의3제2호부터 제4호 까지의...,1974,부칙 제3조 주택 수의 판단 범위에 관한 적용례 제13조의3 제2호부터 제4호까지의...
59998,2 중과제외주택 일반적으로 다주택자는 취득세 중과세하지만 아래의 어느 하나에 해당하...,1974,"2 중과 제외주택 일반적으로 다주택자는 취득세 중과세하지만, 아래의 어느 하나에 해..."


In [5]:
'''맞춤법 검사하기''' 
a = int(len(df_s)/5000) + 1
while a < 38:
    ko_grammar(dfs["df2_%s"%a]['split_str'])
    dfs["df2_%s"%a]['correct_str'] = correct_list
    df_s = pd.concat([df_s,dfs["df2_%s"%a]])
    df_s.to_csv('./산림복지/2020산림복지_본문_맞춤법검사.csv', index=False, encoding='UTF-8')
    a += 1

386


In [ ]:
df_s = pd.read_csv('./산림복지/2020산림복지_본문_맞춤법검사.csv', encoding='UTF-8')
df_s

,split_str,org_idx,correct_str
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...
...,...,...,...
4995,1928년 4월 경북 울진에서 태어나 한때 국민학교 교사 생활을 했던 손씨는 62년...,159,1928년 4월 경북 울진에서 태어나 한때 초등학교 교사 생활을 했던 손 씨는 62...
4996,그러나 공천을 받지 못해 선거에 출마해 보지도 못한 채 정치에의 꿈을 접어야 했고 ...,159,그러나 공천받지 못해 선거에 출마해 보지도 못한 채 정치에의 꿈을 접어야 했고 정치...
4997,손씨는 이후 정치에 미련을 버리고 오대산에 들어가 기도하던 중 신안 을 얻고 나서 ...,159,손 씨는 이후 정치에 미련을 버리고 오대산에 들어가 기도하던 중 신안을 얻고 나서 ...
4998,손씨가 매스컴으로부터 주목 받은 것은 94년 초에 펴낸 터 라는 책에서 김일성의 사...,159,손 씨가 매스컴으로부터 주목받은 것은 94년 초에 펴낸 터라는 책에서 김일성의 사망...
